# EMISSOR chat bot with a BRAIN that interacts through camera and NER

@CLTL

## 1. Introduction

In this demo, we create capsules with triples from inteaction through EMISSOR that are send to the Brain.

This notebook demonstrates a simple chatbot that also takes pictures through your camera while interacting with you

## 2. Installing the platform

virtualenv venv

# install the brain
#pip install --extra-index-url https://test.pypi.org/simple cltl.brain

# install emissor
#%python3 -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple emissor

Alternatively:
* git clone https://github.com/cltl/EMISSOR --branch issue-53-processing
* cd EMISSOR
* python install.py install

# install spacy
#%pip install -U spacy

# install language models
#%python -m spacy download en_core_web_sm

# install cv2
#%pip install opencv-python

## 3. Setting the scene

In [1]:
import emissor
from cltl import brain

In [2]:
import os
import spacy
import time
import uuid
from datetime import datetime
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario
from cltl.brain.long_term_memory import LongTermMemory
from cltl.combot.backend.api.discrete import UtteranceType
import cv2
from datetime import datetime
from datetime import date
from random import getrandbits
import requests

#### The next utils are needed for the interaction and creating triples and capsules
import driver_util as d_util
import capsule_utils as c_util
import text_to_triple as ttt

### 3.1 Creating the scenario and basic values

In [3]:
from random import getrandbits

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
agent = "Leolani2"
human = "Stranger"

### The name of your scenario
scenarioid = "myscenario3"

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "./data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenarioid + "/" + "image"


### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenarioid)
scenario = scenarioStorage.create_scenario(scenarioid, datetime.now().microsecond, datetime.now().microsecond, agent)

Directory  ./data/myscenario3  already exists
Directory  ./data/myscenario3/image  already exists


### 3.2 Adding NERC to a TextSignal as annotations

In [4]:
#### Example of an annotation function that adds annotations to a Signal
#### It adds NERC annotations to the TextSignal and returns a list of entities detected

def add_ner_annotation(signal: TextSignal):
    processor_name = "spaCy"
    utterance = ''.join(signal.seq)

    doc = nlp(utterance)

    offsets, tokens = zip(*[(Index(signal.id, token.idx, token.idx + len(token)), Token.for_string(token.text))
                            for token in doc])

    ents = [NER.for_string(ent.label_) for ent in doc.ents]
    entity_list = [ent.text for ent in doc.ents]
    segments = [token.ruler for token in tokens if token.value in entity_list]

    annotations = [Annotation(AnnotationType.TOKEN.name.lower(), token, processor_name, int(time.time()))
                   for token in tokens]
    ner_annotations = [Annotation(AnnotationType.NER.name.lower(), ent, processor_name, int(time.time()))
                       for ent in ents]

    signal.mentions.extend([Mention(str(uuid.uuid4()), [offset], [annotation])
                            for offset, annotation in zip(offsets, annotations)])
    signal.mentions.extend([Mention(str(uuid.uuid4()), [segment], [annotation])
                            for segment, annotation in zip(segments, ner_annotations)])
    print(entity_list)
    return entity_list

### 3.3 Creating a capsule from the interaction with a triple and perspective

The next function creates a capsule using the Scenario and Signal in combination with values for the perspectives, subject, predicate and object.

In [5]:
def seq_to_text (seq):
    text = ""
    for c in seq:
        text+=c
    return text


def scenario_utterance_to_capsule(scenario: Scenario, signal: TextSignal, author:str, perspective:str, subj: str, pred:str, obj:str):
    place_id = getrandbits(8)
    location = requests.get("https://ipinfo.io").json()

    capsule = {"chat":scenario.id,
                   "turn":signal.id,
                   "author": "carl",
                    "utterance": seq_to_text(signal.seq),
                    "utterance_type": UtteranceType.STATEMENT,
                    "position": "0-"+str(len(signal.seq)),  #TODO generate the true offset range
                    "subject": {"label": subj, "type": "person"},
                    "predicate": {"type": pred},
                    "object":  {"label": obj, "type": "object"},
                    "perspective": perspective ,
                    "context_id": scenario.scenario.context,
                    "date": date.today(),
                    "place": location['city'],
                    "place_id": place_id,
                    "country": location['country'],
                    "region": location['region'],
                    "city": location['city'],
                    "objects": [{'type': 'chair', 'confidence': 0.59, 'id': 1},
                                {'type': 'table', 'confidence': 0.73, 'id': 1},
                               {'type': 'pillbox', 'confidence': 0.32, 'id': 1}],
                            "people": [{'name': 'Carl', 'confidence': 0.98, 'id': 1}]
                  }
    return capsule          

## 4 Starting the interaction

### 4.1 Creating a camera and nlp module

We create a camera and load the NLP model for spaCy.

In [6]:
### Link your camera
camera = cv2.VideoCapture(0)

### Load a language model in spaCy
nlp = spacy.load('en_core_web_sm')

### 4.2 Initialising a BRAIN in GraphDB

Before we start, we need to create an empty Brain or load an existing Brain. The next code assumes we have a repository in GraphDB with the name sandbox as a brain. By setting clear_all=True it is emptied and next loaded with the background ontologies.

In [7]:
import pathlib
log_path=pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                           log_dir=log_path,
                           clear_all=True)

<class 'pathlib.PosixPath'>
2021-09-12 08:46:20,197 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-09-12 08:46:20,198 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Clearing brain
2021-09-12 08:46:22,855 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Checking if ontology is in brain
2021-09-12 08:46:22,858 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-09-12 08:46:23,537 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-09-12 08:46:26,507 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-09-12 08:46:26,510 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-09-12 08:46:26,511 -    DEBUG -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-09-12 08:46:26,513 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-09-12 08:46:26,904 -    DEBUG -   cltl.brain.basic_brain.TrustCalculator - Posting query
2021-09-12 08:46:26,952 -     INF

### 4.3 Starting the CHAT which creates a scenario and saves triples to the BRAIN

In [17]:
##### First signals to get started
success, frame = camera.read()
imagepath = ""
if success:
    imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
    cv2.imwrite(imagepath, frame)

#### Initial prompt by the system from which we create a TextSignal and store it
response = "Hi there. Who are you " + human + "?"
print(agent + ": " + response)
textSignal = d_util.create_text_signal(scenario, response)
scenario.append_signal(textSignal)

utterance = input('\n')
print(human + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)
    # @TODO
    ### Apply some processing to the textSignal and add annotations
    entityText = add_ner_annotation(textSignal)
    scenario.append_signal(textSignal)
    ## Post triples to the brain:

    subj, pred, obj = ttt.getTriplesFromEntities(entityText, textSignal.id)

    response = {}
    if not subj=="":
        print('Subject:', subj, 'Predicate:', pred, 'Object:', obj)
        perspective = {"certainty": 1, "polarity": 1, "sentiment": 1}
        capsule = scenario_utterance_to_capsule(scenario, textSignal, human, perspective, subj, pred, obj)
        #print('Capsule:', capsule)
        response = my_brain.update(capsule, reason_types=True)
        #print(thoughts)
        
    if success:
        imageSignal = d_util.create_image_signal(scenario, imagepath)
        # @TODO
        ### Apply some processing to the imageSignal and add annotations
        ### when done

        scenario.append_signal(imageSignal)

    # Create the response from the system and store this as a new signal
    utterance = ttt.getTextFromTriples(response)
    if not utterance:
        if not entityText:
            utterance = "Any gossip" + '\n'
        else:
            utterance = "So you what do you want to talk about " + entityText[0] + '\n'

    response = utterance[::-1]
    print(agent + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')

    success, frame = camera.read()
    if success:
        imagepath = imagefolder + "/" + str(datetime.now().microsecond) + ".png"
        cv2.imwrite(imagepath, frame)

Leolani2: Hi there. Who are you Stranger?



 Hi there


Stranger: Hi there
[]
Leolani2: Any gossip




 I am Piek


['Piek']
Subject: Piek Predicate: denotedBy Object: c9c0d54d-301a-4537-b38f-0dc16c363247
2021-09-12 08:49:29,304 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-09-12 08:49:29,361 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Posting query
2021-09-12 08:49:29,421 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: piek_denotedby_c9c0d54d-301a-4537-b38f-0dc16c363247 [person_->_object])
2021-09-12 08:49:29,423 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:29,458 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:29,505 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:29,511 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-09-12 08:49:29,513 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:29,558 -    DEBUG -  cltl.brain.bas

...


 I am from Weesp


[]
Leolani2: Any gossip




 Weesp is a little town close to AMsterdam


['AMsterdam']
Subject: AMsterdam Predicate: denotedBy Object: e255deaf-5de3-475a-bd94-eccdc37e5afd
2021-09-12 08:49:53,515 -    DEBUG -    cltl.brain.basic_brain.LongTermMemory - Posting query
2021-09-12 08:49:53,557 -    DEBUG -  cltl.brain.basic_brain.LocationReasoner - Posting query
2021-09-12 08:49:53,615 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: amsterdam_denotedby_e255deaf-5de3-475a-bd94-eccdc37e5afd [person_->_object])
2021-09-12 08:49:53,617 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:53,704 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:53,710 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:53,754 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Entity Novelty: new subject - new object 
2021-09-12 08:49:53,756 -    DEBUG -  cltl.brain.basic_brain.ThoughtGenerator - Posting query
2021-09-12 08:49:53,763 -    DEBUG - 

KeyboardInterrupt: Interrupted by user

### 4.4 Saving the Scenario

In [6]:
scenarioStorage.save_scenario(scenario)

## End of notebook